## 创建模型&数据集

In [ ]:
import torch
import torch.nn as nn

class CustomModel(nn.Module):
    def __init__(self, input_dim=1000, label_feature_dim=10, combined_feature_dim=12):
        super(CustomModel, self).__init__()
        self.input_dim = input_dim
        self.label_feature_dim = label_feature_dim
        self.combined_feature_dim = combined_feature_dim

        # 定义前10维特征提取层
        self.general_feature_extractor = nn.Linear(input_dim, combined_feature_dim - 2)

        # 定义后2维特征提取层
        self.label_specific_extractor = nn.Linear(label_feature_dim, 2)

    def forward(self, x, label_index):
        # 提取前10维特征
        general_features = self.general_feature_extractor(x)

        # 提取与标签相关的特征
        # 假设 label_index 指示了要提取的10维特征的起始位置
        label_specific_features = x[:, label_index:label_index + self.label_feature_dim]
        label_specific_features = self.label_specific_extractor(label_specific_features)

        # 组合特征
        combined_features = torch.cat([general_features, label_specific_features], dim=1)

        return combined_features


In [1]:
import torch
import torch.nn as nn
import protopformer
from torch.nn.init import trunc_normal_

class CustomModel(nn.Module):
    def __init__(self):
        super(CustomModel, self).__init__()
        # 构建 Protopformer 模块
        self.protopformer = protopformer.construct_PPNet(
                            base_architecture='deit_base_patch16_224',
                            pretrained=True, img_size=224,
                            prototype_shape=[1000, 192, 1, 1],
                            num_classes=100,
                            reserve_layers=[11],
                            reserve_token_nums=[81],
                            use_global=True,
                            use_ppc_loss=True,
                            ppc_cov_thresh=1.,
                            ppc_mean_thresh=2.,
                            global_coe=0.5,
                            global_proto_per_class=10,
                            prototype_activation_function='log',
                            add_on_layers_type='regular')

        # # 关闭 Protopformer 参数的梯度
        for param in self.protopformer.parameters():
            param.requires_grad = False
        
        
        # 遍历 protopformer.features.blocks
        # 首先关闭 self.protopformer.features 中所有参数的梯度
        # for param in self.protopformer.features.parameters():
        #     param.requires_grad = False

        # # 然后重新开启最后一个block的梯度
        # for param in self.protopformer.features.blocks[-1].parameters():
        #     param.requires_grad = True


        # 添加额外的层
        self.hash = nn.Linear(1000, 12, bias=False)
        self.variance = nn.Linear(1000, 12, bias=False)
        self.bn_h = nn.BatchNorm1d(12)
        self.bn_v = nn.BatchNorm1d(12)

        # 初始化 hash 和 variance
        self.hash.apply(self._init_weights)
        self.variance.apply(self._init_weights)

    def _init_weights(self, m):
        if isinstance(m, nn.Linear):
            trunc_normal_(m.weight, std=.02)
            if m.bias is not None:
                nn.init.constant_(m.bias, 0)

    def forward(self, images):
        # Protopformer 提取特征
        (cls_tokens, img_tokens), _ = self.protopformer.prototype_distances(images, self.protopformer.reserve_layer_nums)
        local_activations, _ = self.protopformer.get_activations(img_tokens, self.protopformer.prototype_vectors)

        # 应用额外的层
        h = self.bn_h(self.hash(local_activations))
        v = self.bn_v(self.variance(local_activations))
        # h = self.hash(local_activations)
        # v = self.variance(local_activations)
        
        # 应用激活函数和相乘
        v = v / (torch.tanh(v * 1))
        h = torch.tanh(h * 1) 
        x = h * v

        return x, h


In [2]:
import torch

model = CustomModel()

for name, param in model.named_parameters():
    if param.requires_grad:
        print(name)

hash.weight
variance.weight
bn_h.weight
bn_h.bias
bn_v.weight
bn_v.bias


In [3]:
# 要加载的权重文件路径
weight_path = '/wang_hp/zhy/ProtoPFormer/Original+two_phase/output_cosine/CD_CUB2011U/deit_base_patch16_224/1028--adamw-0.05-200-protopformer/checkpoints/epoch-best.pth'
# 加载权重
state_dict = torch.load(weight_path, map_location='cpu')
# 导入权重到模型

model.protopformer.load_state_dict(state_dict['model'])

<All keys matched successfully>

### 准备数据集

In [4]:
from tools.datasets import build_dataset

class Args:
    color_jitter = 0.4
    aa = 'rand-m9-mstd0.5-inc1'
    smoothing = 0.0
    train_interpolation = 'bicubic'
    reprob = 0.25
    remode = 'pixel'
    recount = 1
    resplit = False
    input_size = 224
    data_set = 'CD_CUB2011U'
    batch_size = 256
    # 添加其他必需的参数

args = Args()


dataset_train, dataset_val, train_dataset_unlabelled, nb_classes = build_dataset(is_train=True, args=args)
print("Dataset num_classes: {}".format(nb_classes))
print("train {} test: {}".format(len(dataset_train), len(dataset_val)))
print("train_dataset_unlabelled {}".format(len(train_dataset_unlabelled)))

/opt/conda/envs/protop/lib/python3.8/site-packages/torchvision/transforms/transforms.py:257: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  warnings.warn(


Files already downloaded and verified
Dataset num_classes: 100
train 1500 test: 2864
train_dataset_unlabelled 4494


## 训练和测试

In [5]:
class SupConLoss(torch.nn.Module):
    """Supervised Contrastive Learning: https://arxiv.org/pdf/2004.11362.pdf.
    It also supports the unsupervised contrastive loss in SimCLR
    From: https://github.com/HobbitLong/SupContrast"""
    def __init__(self, temperature=0.07, contrast_mode='all',
                 base_temperature=0.07):
        super(SupConLoss, self).__init__()
        self.temperature = temperature
        self.contrast_mode = contrast_mode
        self.base_temperature = base_temperature

    def forward(self, features, labels=None, mask=None):
        """Compute loss for model. If both `labels` and `mask` are None,
        it degenerates to SimCLR unsupervised loss:
        https://arxiv.org/pdf/2002.05709.pdf
        Args:
            features: hidden vector of shape [bsz, n_views, ...].
            labels: ground truth of shape [bsz].
            mask: contrastive mask of shape [bsz, bsz], mask_{i,j}=1 if sample j
                has the same class as sample i. Can be asymmetric.
        Returns:
            A loss scalar.
        """

        device = (torch.device('cuda')
                  if features.is_cuda
                  else torch.device('cpu'))

        if len(features.shape) < 3:
            raise ValueError('`features` needs to be [bsz, n_views, ...],'
                             'at least 3 dimensions are required')
        if len(features.shape) > 3:
            features = features.view(features.shape[0], features.shape[1], -1)

        batch_size = features.shape[0]
        if labels is not None and mask is not None:
            raise ValueError('Cannot define both `labels` and `mask`')
        elif labels is None and mask is None:
            mask = torch.eye(batch_size, dtype=torch.float32).to(device)
        elif labels is not None:
            labels = labels.contiguous().view(-1, 1)
            if labels.shape[0] != batch_size:
                raise ValueError('Num of labels does not match num of features')
            mask = torch.eq(labels, labels.T).float().to(device)
        else:
            mask = mask.float().to(device)

        contrast_count = features.shape[1]
        contrast_feature = torch.cat(torch.unbind(features, dim=1), dim=0)
        if self.contrast_mode == 'one':
            anchor_feature = features[:, 0]
            anchor_count = 1
        elif self.contrast_mode == 'all':
            anchor_feature = contrast_feature
            anchor_count = contrast_count
        else:
            raise ValueError('Unknown mode: {}'.format(self.contrast_mode))

        # compute logits
        anchor_dot_contrast = torch.div(
            torch.matmul(anchor_feature, contrast_feature.T),
            self.temperature)

        # for numerical stability
        logits_max, _ = torch.max(anchor_dot_contrast, dim=1, keepdim=True)
        logits = anchor_dot_contrast - logits_max.detach()

        # tile mask
        mask = mask.repeat(anchor_count, contrast_count)
        # mask-out self-contrast cases
        logits_mask = torch.scatter(
            torch.ones_like(mask),
            1,
            torch.arange(batch_size * anchor_count).view(-1, 1).to(device),
            0
        )
        mask = mask * logits_mask

        # compute log_prob
        exp_logits = torch.exp(logits) * logits_mask
        log_prob = logits - torch.log(exp_logits.sum(1, keepdim=True))

        # compute mean of log-likelihood over positive
        mean_log_prob_pos = (mask * log_prob).sum(1) / mask.sum(1)

        # loss
        loss = - (self.temperature / self.base_temperature) * mean_log_prob_pos
        loss = loss.view(anchor_count, batch_size).mean()

        return loss

In [6]:
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):

        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):

        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [7]:
from torch.optim import AdamW, SGD
from torch.optim.lr_scheduler import CosineAnnealingLR
from tqdm import tqdm
import numpy as np
from scipy.optimize import linear_sum_assignment as linear_assignment

class Args_train:
    lr = 0.1
    momentum = 0.9
    weight_decay = 1e-4
    epochs = 200
    num_workers = 8
    batch_size = 128
    device = 'cuda'
    lr_protopformer = 1e-4
    lr_others = 0.01
    # 添加其他必需的参数

args_train = Args_train()



def train(model, train_loader, unlabelled_train_loader, args):
    # 过滤出 model 中需要梯度的参数
    params_to_optimize = [param for param in model.parameters() if param.requires_grad]

    # 创建优化器，仅针对需要梯度的参数
    optimizer = SGD(params_to_optimize, lr=args.lr, momentum=args.momentum, weight_decay=args.weight_decay)

    exp_lr_scheduler = CosineAnnealingLR(
            optimizer,
            T_max=args.epochs,
            eta_min=args.lr * 1e-3,
        )
    
    # # 分离 Protopformer 的参数和其他参数
    # protopformer_params = [param for param in model.protopformer.parameters() if param.requires_grad]
    # other_params = [param for name, param in model.named_parameters() if "protopformer" not in name and param.requires_grad]
    # # 创建优化器，为不同的参数组设置不同的学习率
    # optimizer = AdamW([
    #     {'params': protopformer_params, 'lr': args.lr_protopformer},
    #     {'params': other_params, 'lr': args.lr_others}
    # ])
    # exp_lr_scheduler = CosineAnnealingLR(
    #         optimizer,
    #         T_max=args.epochs)

    sup_con_crit = SupConLoss()
    best_test_acc_lab = 0

    for epoch in range(args.epochs):

        loss_record = AverageMeter()
        train_acc_record = AverageMeter()

        model.train()

        for batch_idx, batch in enumerate(tqdm(train_loader)):

            images, class_labels, uq_idxs = batch

            class_labels = class_labels.to(args.device)
            images = torch.cat(images, dim=0).to(args.device)

            features, hash_features = model(images)
            
            features = torch.nn.functional.normalize(features, dim=-1)

            f1, f2 = [f for f in features.chunk(2)]
            sup_con_feats = torch.cat([f1.unsqueeze(1), f2.unsqueeze(1)], dim=1)
            sup_con_labels = class_labels
            sup_con_loss = sup_con_crit(sup_con_feats, labels=sup_con_labels)
            
            reg_loss = (1 - torch.abs(hash_features)).mean()
            
            loss = sup_con_loss * 1 + reg_loss * 3

            loss_record.update(loss.item(), class_labels.size(0))
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()


        print('Train Epoch: {} Avg Loss: {:.4f} | Seen Class Acc: {:.4f} '.format(epoch, loss_record.avg,
                                                                                  train_acc_record.avg))


        with torch.no_grad():
            all_acc, old_acc, new_acc = test_on_the_fly(model, unlabelled_train_loader)


        print('Train Accuracies: All {:.4f} | Old {:.4f} | New {:.4f}'.format(all_acc, old_acc,
                                                                              new_acc))

        # Step schedule
        exp_lr_scheduler.step()


        if all_acc > best_test_acc_lab:

            print(f'Best ACC on old Classes on disjoint test set: {all_acc:.4f}...')
            print('Best Train Accuracies: All {:.4f} | Old {:.4f} | New {:.4f}'.format(all_acc, old_acc,
                                                                                  new_acc))


            best_test_acc_lab = all_acc


def split_cluster_acc_v2(y_true, y_pred, mask):
    """
    Calculate clustering accuracy. Require scikit-learn installed
    First compute linear assignment on all data, then look at how good the accuracy is on subsets

    # Arguments
        mask: Which instances come from old classes (True) and which ones come from new classes (False)
        y: true labels, numpy.array with shape `(n_samples,)`
        y_pred: predicted labels, numpy.array with shape `(n_samples,)`

    # Return
        accuracy, in [0,1]
    """
    mask = mask.astype(bool)
    y_true = y_true.astype(int)
    y_pred = y_pred.astype(int)

    old_classes_gt = set(y_true[mask])
    new_classes_gt = set(y_true[~mask])

    assert y_pred.size == y_true.size
    D = max(y_pred.max(), y_true.max()) + 1
    w = np.zeros((D, D), dtype=int)
    for i in range(y_pred.size):
        w[y_pred[i], y_true[i]] += 1

    ind = linear_assignment(w.max() - w)
    ind = np.vstack(ind).T

    ind_map = {j: i for i, j in ind}
    total_acc = sum([w[i, j] for i, j in ind]) * 1.0 / y_pred.size

    old_acc = 0
    total_old_instances = 0
    for i in old_classes_gt:
        old_acc += w[ind_map[i], i]
        total_old_instances += sum(w[:, i])
    old_acc /= total_old_instances

    new_acc = 0
    total_new_instances = 0
    for i in new_classes_gt:
        new_acc += w[ind_map[i], i]
        total_new_instances += sum(w[:, i])
    new_acc /= total_new_instances

    return total_acc, old_acc, new_acc

def test_on_the_fly(model, test_loader):

    model.eval()

    all_feats = []
    targets = np.array([])
    mask = np.array([])

    print('Collating features...')
    # First extract all features
    for batch_idx, (images, label, _) in enumerate(test_loader):

        images = images.cuda()

        # Pass features through base model and then additional learnable transform (linear layer)
        with torch.no_grad():
            _, hash_feats = model(images)


        all_feats.append(hash_feats.cpu().numpy())
        targets = np.append(targets, label.cpu().numpy())
        mask = np.append(mask, np.array([True if x.item() in range(100)
                                         else False for x in label]))

    # -----------------------
    # On-The-Fly
    # -----------------------
    all_feats = np.concatenate(all_feats)
    feats_hash = torch.Tensor(all_feats > 0).float().tolist()
    preds = []
    hash_dict = []
    for feat in feats_hash:
        if not feat in hash_dict:
            hash_dict.append(feat)
        preds.append(hash_dict.index(feat))
    preds = np.array(preds)
    print(len(list(set(preds))), len(preds))

    # -----------------------
    # EVALUATE
    # -----------------------
    all_acc, old_acc, new_acc = split_cluster_acc_v2(y_true=targets, y_pred=preds, mask=mask)

    return all_acc, old_acc, new_acc

In [8]:
from torch.utils.data import DataLoader
model = model.to('cuda')
labelled_train_loader = DataLoader(dataset_train, num_workers=args_train.num_workers, batch_size=args_train.batch_size, 
                            shuffle=True, drop_last=True)
unlabelled_train_loader = DataLoader(train_dataset_unlabelled, num_workers=args_train.num_workers,
                                    batch_size=args_train.batch_size, shuffle=False)
train(model, labelled_train_loader, unlabelled_train_loader, args_train)

100%|██████████| 11/11 [00:19<00:00,  1.79s/it]

Train Epoch: 0 Avg Loss: 5.1064 | Seen Class Acc: 0.0000 
Collating features...


46 4494
Train Accuracies: All 0.0850 | Old 0.1173 | New 0.0688
Best ACC on old Classes on disjoint test set: 0.0850...
Best Train Accuracies: All 0.0850 | Old 0.1173 | New 0.0688


100%|██████████| 11/11 [00:15<00:00,  1.42s/it]

Train Epoch: 1 Avg Loss: 3.9118 | Seen Class Acc: 0.0000 
Collating features...


...

74 4494
Train Accuracies: All 0.1077 | Old 0.1500 | New 0.0865


100%|██████████| 11/11 [00:21<00:00,  1.94s/it]

Train Epoch: 199 Avg Loss: 1.9115 | Seen Class Acc: 0.0000 
Collating features...


73 4494
Train Accuracies: All 0.1079 | Old 0.1407 | New 0.0915


: 